## **Import important Libraries**

In [1]:
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

In [ ]:
dataset = torchvision.datasets.MNIST(
    root='./data', transform=torchvision.transforms.ToTensor()
)

Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet
complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html

On Images
- CenterCrop, Grayscale, Pad, RandomAffine
- RandomCrop, RandomHorizontalFlip, RandomRotation
- Resize, Scale

On Tensors
- LinearTransformation, Normalize, RandomErasing

Conversion
- ToPILImage: from tensor or ndrarray
- ToTensor : from numpy.ndarray or PILImage

Generic
- Use Lambda 

Custom
- Write own class

Compose multiple Transforms
- composed = transforms.Compose([Rescale(256), RandomCrop(224)])

### **Add Transform to dataset**

In [4]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        # data loading
        xy = np.loadtxt('./data/wine/wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        # do not convert to tensor data
        self.x = xy[:, 1:]
        self.y = xy[:, [0]]

        self.transform = transform

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]

        if self.transform:
            sample = self.transform(sample)
        
        return sample

    def __len__(self):
        return self.n_samples

### **Transform Class for converting to tensor**

In [5]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

#### **Pass transform to dataset**

In [6]:
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


#### **Not pass transform to dataset**

In [10]:
dataset = WineDataset(transform=None)
first_data = dataset[0]
features, labels = first_data
print(features, labels)
print(type(features), type(labels))

[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03] [1.]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


### **Custom Transform**

In [8]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

### **Compose Transform**

In [11]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(features, labels)
print(type(features), type(labels))

tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]) tensor([1.])
<class 'torch.Tensor'> <class 'torch.Tensor'>
